# GHZ Game

The **GHZ Game** quantum kata is a series of exercises designed
to get you familiar with the GHZ game.

In it three players (Alice, Bob and Charlie) try to win the following game:

Each of them is given a bit (r, s and t respectively), and
they have to return new bits (a, b and c respectively) so
that  r ∨ s ∨ t = a ⊕ b ⊕ c.* The input bits will have
zero or two bits set to true and three or one bits set to false.
The trick is, the players can not communicate during the game.

```
* '∨' is the standard bitwise OR operator.
* '⊕' is the exclusive or, or XOR operator, so 'P ⊕ Q' is true if exactly one of P and Q is true.
```

* You can read more about the GHZ game in the [lecture notes](https://cs.uwaterloo.ca/~watrous/CPSC519/LectureNotes/20.pdf) by John Watrous. 
* Another description can be found in the [lecture notes](https://staff.fnwi.uva.nl/m.walter/physics491/lecture1.pdf) by Michael Walter.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl/⌘+Enter.


To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [1]:
%package Microsoft.Quantum.Katas::0.12.20072031

Adding package Microsoft.Quantum.Katas::0.12.20072031: done!

Microsoft.Quantum.Standard::0.12.20072031, Microsoft.Quantum.Katas::0.12.20072031

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## Part I. Classical GHZ


### Task 1.1. Win Condition
**Inputs:** 

  1. Alice, Bob and Charlie's input bits (r, s and t), stored as an array of length 3,

  2. Alice, Bob and Charlie's output bits (a, b and c), stored as an array of length 3.

The input bits will have zero or two bits set to true.

**Output:** 
True if Alice, Bob and Charlie won the GHZ game, that is, if r ∨ s ∨ t = a ⊕ b ⊕ c, and false otherwise.

In [2]:
%kata T11_WinCondition_Test 

open Microsoft.Quantum.Logical;

function WinCondition (rst : Bool[], abc : Bool[]) : Bool {
    // There are four inputs possible, (0,0,0), (0,1,1), (1,0,1), and (1,1,0), each with 25% probability.
    // Therefore, in order to win, the sum of the output bits has to be even if the input is (0,0,0) and odd otherwise.
    
    let p = Or(rst[0], Or(rst[1], rst[2])); 
    let u = Xor(abc[0], Xor(abc[1], abc[2]));
    return (p == u);
}

Success!

### Task 1.2. Random classical strategy

**Input:** The input bit for one of the players (r, s or t).

**Output:** A random bit that this player will output (a, b or c).

If all players use this strategy, they will win about 50% of the time.

In [3]:
%kata T12_RandomClassical_Test 

operation RandomClassicalStrategy (input : Bool) : Bool {
    let coin = Random([0.5, 0.5]);
    return (coin == 1);
}

Success!

### Task 1.3. Best classical strategy

**Input:** The input bit for one of the players (r, s or t).

**Output:** A bit that this player will output (a, b or c) to maximize their chance of winning.

All players will use the same strategy.
The best classical strategy should win about 75% of the time.

In [4]:
%kata T13_BestClassical_Test 

operation BestClassicalStrategy (input : Bool) : Bool {
    // If all three players return TRUE, then a ⊕ b ⊕ c = TRUE by necessity (since the sum of their bits is odd).
    // This will win against inputs of |011>, |101>, and |110> and lose against |000>.
    // Since the four above inputs have equal probability and represent all possible inputs,
    // this strategy wins with 75% probability.
    return true;
}

Success!

### Task 1.4. Referee classical GHZ game

**Inputs:** 

  1. An operation which implements a classical strategy (i.e., takes an input bit and produces an output bit),

  2. An array of 3 input bits that should be passed to the players.

**Output:** 
An array of 3 bits that will be produced if each player uses this strategy.

In [5]:
%kata T14_PlayClassicalGHZ_Test 

operation PlayClassicalGHZ (strategy : (Bool => Bool), inputs : Bool[]) : Bool[] {
    // Input bits and strategy are given, so simply convert them to the output bits.
    let r = inputs[0];
    let s = inputs[1];
    let t = inputs[2];
    let a = strategy(r);
    let b = strategy(s);
    let c = strategy(t);
    return [a, b, c];
}

Success!

## Part II. Quantum GHZ

In the quantum version of the game, the players still can not
communicate during the game, but they are allowed to share 
qubits from an entangled triple before the start of the game.


### Task 2.1. Entangled triple

**Input:** An array of three qubits in the $|000\rangle$ state.

**Goal:** Create the entangled state $|\Phi\rangle = \frac{1}{2} \big(|000\rangle - |011\rangle - |101\rangle - |110\rangle \big)$ on these qubits.

In [6]:
%kata T21_CreateEntangledTriple_Test 

operation CreateEntangledTriple (qs : Qubit[]) : Unit {
    X(qs[0]);
    X(qs[1]);
    // X ⊗ X is a matrix with 1's on the top-right to bottom-left diagonal and zero otherwise,
    // so the above leads to |110>.
    // An alternative way to look at this is simply as a pair of NOT operations on the first two qubits.

    H(qs[0]);
    H(qs[1]);
    // As the right-hand column of H ⊗ H goes [+1/2, -1/2, -1/2, +1/2],
    // the above leads to 1/2 (|000> - |010> - |100> + |110>).
    // See:  https://en.wikipedia.org/wiki/Hadamard_transform  for more info.

    CZ(qs[0], qs[1]);
    // CZ (controlled Z) is a 4x4 matrix with +1, +1, +1, -1 along the diagonal and otherwise zero,
    // so the above leads to 1/2 (|000> - |010> - |100> - |110>).

    (ControlledOnBitString([false, true], X))([qs[0], qs[1]], qs[2]);
    (ControlledOnBitString([true, false], X))([qs[0], qs[1]], qs[2]);
    // Applies a NOT gate to qubit 3 if and only if first two qubits match the given bit mask,
    // so now we have our desired state: 1/2 (|000> - |011> - |101> - |110>).
}

Success!

### Task 2.2. Quantum strategy

**Inputs:**

  1. The input bit for one of the players (r, s or t),

  2. That player's qubit of the entangled triple shared between the players.

**Goal:** Measure the qubit in the Z basis if the bit is 0 (false), or the X basis if the bit is 1 (true), and return the result.

The state of the qubit after the operation does not matter.

In [7]:
%kata T22_QuantumStrategy_Test 

open Microsoft.Quantum.Measurement;

operation QuantumStrategy (input : Bool, qubit : Qubit) : Bool {
    if (input) {
        let q = MResetX(qubit);
        return (q == One);
    }
    else {
        let q = MResetZ(qubit);
        return (q == One);
    }
}

Success!

### Why the above strategy has a 100% win rate

Recall the formula for the win condition:
* The sum of the answer bits must be even if the question bits are (0,0,0)
* The sum of the answer bits must be odd if the question bits are (1,1,0), (1,0,1) or (0,1,1).

Now, in standard tensor expansion, the probability "wavefunction" for three qubits is given by the following vector of length 8:

$
\begin{bmatrix}
\psi_{000}\\
\psi_{001}\\
\psi_{010}\\
\psi_{011}\\
\psi_{100}\\
\psi_{101}\\
\psi_{110}\\
\psi_{111}
\end{bmatrix}
$

where $\psi$ represents the "wavefunction" amplitude for the three-qubit combination.  $\psi^2$ gives the probability of observing that combination upon measurement.

Now, for the entangled state $|\Phi\rangle$ that Alice, Bob and Charlie have agreed to use, this vector has the values

$
\begin{bmatrix}
+1/2\\
 0\\
 0\\
-1/2\\
 0\\
-1/2\\
-1/2\\
 0
\end{bmatrix}
$

To win, their answers must (but need only) match the odd-even parity in the event of three zeroes and be opposite otherwise.  So for three zeros, it suffices for all three players to take a standard Z basis measurement, all of which return one of the four original combinations from the entangled state, which sum to an even number and the team wins.  Another way to see this is the form of the $Z \otimes Z \otimes Z$ tensor product

$
Z \otimes Z \otimes Z = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & -1 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & -1 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & -1 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & -1
\end{bmatrix}:  
$
converts $(+a, +b, +c, +d, +e, +f, +g, +h)$ to $(+a, -b, -c, +d, -e, +f, +g, -h)$.

As $a$, $d$, $f$ and $g$ are the only terms with any amplitude, the parity is preserved and the team wins.

Now, suppose the team got $|110\rangle$.  Charlie, looking at a zero, takes a Z basis measurement as before, while Alice and Bob each take X basis measurements.  (An X basis measurement is also equivalent to performing a Hadamard transform followed by a standard Z basis measurement, as the X basis runs along the $| + \rangle$ / $| - \rangle$ axis and a Hadamard transform rotates the $| 0 \rangle$ / $| 1 \rangle$ polarity to $| + \rangle$ / $| - \rangle$.)  One of three tensor products with a Z and two X units will be utilized, all with the same effect:

$
X \otimes X \otimes Z = \begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & -1\\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & -1 & 0 & 0\\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & -1 & 0 & 0 & 0 & 0\\
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & -1 & 0 & 0 & 0 & 0 & 0 & 0
\end{bmatrix}:
$
converts $(+a, +b, +c, +d, +e, +f, +g, +h)$ to $(+g, -h, +e, -f, +c, -d, +a, -b)$.

$
X \otimes Z \otimes X = \begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & -1\\
0 & 0 & 0 & 0 & 0 & 0 & -1 & 0\\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & -1 & 0 & 0 & 0 & 0\\
0 & 0 & -1 & 0 & 0 & 0 & 0 & 0
\end{bmatrix}:
$
converts $(+a, +b, +c, +d, +e, +f, +g, +h)$ to $(+f, +e, -h, -g, +b, +a, -d, -c)$.

$
Z \otimes X \otimes X = \begin{bmatrix}
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & -1\\
0 & 0 & 0 & 0 & 0 & 0 & -1 & 0\\
0 & 0 & 0 & 0 & 0 & -1 & 0 & 0\\
0 & 0 & 0 & 0 & -1 & 0 & 0 & 0
\end{bmatrix}:
$
converts $(+a, +b, +c, +d, +e, +f, +g, +h)$ to $(+d, +c, +b, +a, -h, -g, -f, -e)$.

Again, the only terms that matter are $a$, $d$, $f$ and $g$.  All three of the above $8 \times 8$ tensor products swap $a$ (which always has a $+1/2$ value) with one of $d$, $f$, or $g$ (all having a $-1/2$ value), while the other two are replaced by each other's additive inverses (as both are $-1/2$, this has the effect of changing both to $+1/2$).  The upshot of this is that the two Hadamard transforms multiply the original $\psi$ vector by $-1$, essentially reversing the parity of the team's entangled state and giving them their win.

Another way to look at it is that two Hadamard transforms (one performed by each player holding a 1 bit) will convert the original entangled state into one where the combinations are $|001\rangle$, $|010\rangle$, $|100\rangle$, and $|111\rangle$ with equal probability and the team will always win.  The entanglement of the qubits makes the maneuver work.

### Task 2.3. Play the GHZ game using the quantum strategy

**Input:** Operations that return Alice, Bob and Charlie's output bits (a, b and c) based on
their quantum strategies and given their respective qubits from the entangled triple.
The players have already been told what their starting bits (r, s and t) are.

**Goal:** Return an array of players' output bits (a, b and c).

In [8]:
%kata T23_PlayQuantumGHZ_Test 

operation PlayQuantumGHZ (strategies : (Qubit => Bool)[]) : Bool[] {
    using (ghz = Qubit[3]) {
        CreateEntangledTriple(ghz);

        let a = strategies[0](ghz[0]);
        let b = strategies[1](ghz[1]);
        let c = strategies[2](ghz[2]);

        return [a, b, c];
    }
}

Success!